In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Directories
DATASET_DIR = "dataset"
MODEL_PATH = "face_recognition_model.h5"
CLASS_INDICES_PATH = "class_indices.json"

# Ensure dataset directory exists
os.makedirs(DATASET_DIR, exist_ok=True)

def capture_images(user_name):
    user_dir = os.path.join(DATASET_DIR, user_name)
    os.makedirs(user_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(0)
    print("Press 'q' to quit capturing.")
    count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break
        
        cv2.imshow("Capture Face", frame)
        filepath = os.path.join(user_dir, f"{count}.jpg")
        cv2.imwrite(filepath, frame)
        count += 1
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Captured {count} images for user: {user_name}")

def preprocess_data():
    # Applying Data Augmentation
    datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.2
    )
    
    train_generator = datagen.flow_from_directory(
        DATASET_DIR,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    
    val_generator = datagen.flow_from_directory(
        DATASET_DIR,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_generator, val_generator

def build_model(num_classes):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model():
    train_generator, val_generator = preprocess_data()
    num_classes = len(train_generator.class_indices)
    
    model = build_model(num_classes)
    model.fit(train_generator, epochs=20, validation_data=val_generator)
    model.save(MODEL_PATH)
    print(f"Model saved to {MODEL_PATH}")
    
    # Save class indices
    with open(CLASS_INDICES_PATH, 'w') as f:
        json.dump(train_generator.class_indices, f)
    print(f"Class indices saved to {CLASS_INDICES_PATH}")

def delete_user(user_name):
    user_dir = os.path.join(DATASET_DIR, user_name)
    if os.path.exists(user_dir):
        for file in os.listdir(user_dir):
            os.remove(os.path.join(user_dir, file))
        os.rmdir(user_dir)
        print(f"Deleted data for user: {user_name}")
    else:
        print(f"User {user_name} not found.")

if __name__ == "__main__":
    while True:
        print("\n1. Capture Images")
        print("2. Train Model")
        print("3. Delete User")
        print("4. Quit")
        choice = input("Enter your choice (1-4): ").strip()
        
        if choice == '1':
            user_name = input("Enter user name: ").strip()
            capture_images(user_name)
        elif choice == '2':
            train_model()
        elif choice == '3':
            user_name = input("Enter user name to delete: ").strip()
            delete_user(user_name)
        elif choice == '4':
            break
        else:
            print("Invalid choice. Please try again.")



1. Capture Images
2. Train Model
3. Delete User
4. Quit
